# Applicative Functors

Remember that a functor in programming is class of parametrized types with an `fmap`, e.g.
for a struct `F{T}`, we have `fmap(f::Function, x::F)` that *lifts* a function on
type `T` to type `M` to a function on `F{T}` to `F{M}`.

In [1]:
using Pkg
Pkg.activate(".")
using PartialFunctions

include("Currier.jl");
using .Currier

  Activating project at `~/MEGA/EMAP/Julia_Tutorials/CategoryTheory`


In [2]:
struct SimpleFunctor{T}
    _1::T
end
fmap(f::Function, x::SimpleFunctor) = SimpleFunctor(f(x._1))
fmap(x::Int-> x + 1, SimpleFunctor(10))

SimpleFunctor{Int64}(11)

We can expand on this idea of fmaps in the following way. Consider
a function that takes two arguments, e.g `+`. Our `fmap` only
lifts functions with "single" arguments, such that a function
acting of a type `T` is lifted to act on type `F{T}`.
What we want to do is to be able to do is lift functions with many arguments.
For example:

In [3]:
fmap2(f::Function, x1::SimpleFunctor, x2::SimpleFunctor) = SimpleFunctor(f(x1._1,x2._1))
fmap2(+, SimpleFunctor(1), SimpleFunctor(2))

SimpleFunctor{Int64}(3)

Our implementation works. What we want is to expand it to any number of arguments...
For example, `fmap0` should take a function with zero arguments (i.e. a value).

Let's implement this:

In [4]:
fmap0(v, ::Type{SimpleFunctor}) = SimpleFunctor(v)

fmap0(10, SimpleFunctor)

SimpleFunctor{Int64}(10)

A parametric struct that has this generalized `fmap` is an applicative functor.
In order to construct all these `fmap0`,  `fmap1`, ... we only need
to define two functions:
```haskell
pure :: a -> F a
(<*>) :: F (a -> b) -> F a -> F b
```

## Example - Maybe Applicative

In [5]:
struct Just{T}
    _1::T
end
struct Nothing end
Maybe{T} = Union{Just{T}, Nothing};

fmap(f::Function, x::Nothing) = Nothing()
fmap(f::Function, x::Just)    = Just(f(x._1))


fmap (generic function with 3 methods)

In [6]:
pure(x,::Type{SimpleFunctor}) = Just(x)
⊗(mg::Nothing,mx::Maybe) = Nothing()
⊗(mg::Just{<:Function},mx::Maybe) = fmap(mg._1, mx)

pure(10,SimpleFunctor)

Just{Int64}(10)

In [7]:
⊗(Nothing(), Just(10))

Nothing()

In [8]:
⊗(Just(x->x+1), Just(10))

Just{Int64}(11)

In [9]:
pure(x->x+1,SimpleFunctor) ⊗ Just(1)

Just{Int64}(2)

We almost got it right. But note that the example below fails: 

In [10]:
pure((x,y)->x+y, SimpleFunctor) ⊗ Just(10) ⊗ Just(10)

LoadError: MethodError: no method matching (::var"#11#12")(::Int64)

[0mClosest candidates are:
[0m  (::var"#11#12")(::Any, [91m::Any[39m)
[0m[90m   @[39m [35mMain[39m [90m[4mIn[10]:1[24m[39m


**We have a problem!**
Our implementation worked for the previous examples, but it does not actually work
when our functions have more than one argument. Why?
Because Julia does not have native currying like Haskell. This means,
for example:
```julia
f(x,y) = 2*(x + y)

f(10,_) #This does not work!
```
What we want is to be able to apply a single argument to a function, and
later on apply the other arguments.

Julia does have lambda functions, so we could construct
`y->f(10,y)`, which would be the curried version for `f(10,_)`.
Here, we use the `PartialFunctions.jl` package, which implements macros
that do this partial application for us.

There is one issue with partial functions that they do not immediatly apply the function, e.g.
```julia
f $ 1 $ 2 #returns a function z -> f(1,2...)
```
Thus, if we want to actually run this function, we need to actually store and run it, or
we need to pass the last argument with `<|`, e.g.

```julia
f $ 1 <| 2      # returns 3
f $ 1 $ 2 <| () # returns 3

h = f $ 1 $ 2
h()             # returns 3
```

Therefore, our applicative will also need a way to define whether the number of
argumets for a function has "saturated". Our solution was to pass `()` in the end of the applicative.

In [11]:
fmap(f::Function, x::Nothing) = Nothing()
fmap(f::Function, x::Just)    = Just(f $ x._1)
fmap(::Tuple{}, x::Just)      = Just(x._1())
fmap(::Tuple{}, x::Nothing)   = Nothing()

pure(x,::Type{SimpleFunctor}) = Just(x)
⊗(mg::Nothing,mx::Maybe) = Nothing()
⊗(mg::Just{<:Function},mx::Maybe) = fmap(mg._1, mx)
⊗(mg::Maybe,::Tuple{}) = fmap((),mg)

⊗ (generic function with 3 methods)

In [12]:
g = pure((x,y)->x+y, SimpleFunctor) ⊗ Just(10) ⊗ Just(10) ⊗ ()

h = pure((x,y,z)->z*(x+y), SimpleFunctor) ⊗ Just(10) ⊗ Just(10) ⊗ Just(2) ⊗ ()

l = pure((x,y,z)->z*(x+y), SimpleFunctor) ⊗ Just(10) ⊗ Nothing() ⊗ Just(2) ⊗ ()

g,h,l

(Just{Int64}(20), Just{Int64}(40), Nothing())

## Example - List Applicative

We start by defining our List functor.

In [13]:
struct Nil end
struct Cons{T}
  val::T
  next::Union{Cons{T},Nil} # n.b. abstract type!
end
List{T} = Union{Cons{T}, Nil}
fmap(f::Function, x::Nil) = Nil()
fmap(f::Function, x::Cons) = Cons(f(x.val),fmap(f, x.next))

fmap(x->x+1, Cons(1,Cons(2,Nil())))

Cons{Int64}(2, Cons{Int64}(3, Nil()))

To make it into an applicative, we need to enable the partial
function application. Thus, we need to modify our fmap.

In [14]:
fmap(f::Function, x::Cons) = Cons(f $ x.val,fmap(f, x.next))
fmap(::Tuple{}, x::Nil)    = Nil()
fmap(::Tuple{}, x::Cons)   = Cons(x.val(), fmap((),x.next))

fmap(x->x+1, Cons(1,Cons(2,Nil()))).val()

2

To properly define the ⊗ operator, we need a concatenation function for our List struct.

In [15]:
concatenate(x::Nil ,y::List) = y
concatenate(x::Cons,y::List) = Cons(x.val, concatenate(x.next, y))

concatenate (generic function with 2 methods)

Finally, we can construct our applicative. 

In [35]:
pure(x, ::Type{List}) = Cons(x, Nil())

⊗(mg::Nil,mx::List) = Nil()
⊗(mg::Cons{<:Function},mx::List) = fmap(mg.val, mx)
⊗(mg::Cons{<:Function},mx::List) = concatenate(fmap(mg.val, mx), mg.next ⊗ mx)
⊗(mg::List,::Tuple{}) = fmap((),mg)

⊗ (generic function with 6 methods)

In [46]:
println(pure(x->x+1, List) ⊗ Cons(1,Nil()) ⊗ () )
println(pure(+, List) ⊗ Cons(1,Nil()) ⊗ Cons(2,Nil()) ⊗ () )
println(pure(*, List) ⊗ Cons(1,Cons(2,Nil())) ⊗ Cons(3,Cons(4,Nil())) ⊗ () )

Cons{Int64}(2, Nil())
Cons{Int64}(3, Nil())
Cons{Int64}(3, Cons{Int64}(4, Cons{Int64}(6, Cons{Int64}(8, Nil()))))
